# Asignación de tutores pares

**Alumno:** VASQUEZ RAMOS, Jose Manuel

**Fecha:** 20/05/2025

___

## Contexto académico & objetivo

Dispones de una tabla “disponibilidad” (filas = estudiantes mentores, columnas = horarios). Minimiza el número de choques de horario asignando a cada mentor un bloque de 2 h.

## Indicaciones clave

Representa cada solución como lista de horarios. Función de costo = choques. Vecindad: cambiar 1 horario. Devuelve horario final y choques = 0 si posible.

## Librerías

In [1]:
import pandas as pd
import numpy as np
import random

## Control de aleatoriedad

In [2]:
random.seed(42)
np.random.seed(42)

## Leer disponibilidad

In [9]:
df = pd.read_csv("dataset/mentorAvailability.csv")
mentores = df['MentorID'].tolist()
slots = df.columns[1:]  # Slot1 a Slot10
num_slots = len(slots)

## Funciones auxiliares

In [10]:
def get_valid_blocks(mentor_row):
    """Devuelve los bloques de 2h continuos disponibles para un mentor."""
    disponibilidad = mentor_row.values
    bloques = []
    for i in range(num_slots - 1):
        if disponibilidad[i] == 1 and disponibilidad[i + 1] == 1:
            bloques.append(i)
    return bloques  # índice del primer slot

In [11]:
def initial_solution(df):
    """Genera una solución inicial aleatoria válida (un bloque por mentor)."""
    asignacion = {}
    for idx, row in df.iterrows():
        posibles = get_valid_blocks(row[1:])
        if posibles:
            asignacion[row['MentorID']] = random.choice(posibles)
        else:
            asignacion[row['MentorID']] = None  # No tiene bloque válido
    return asignacion

In [12]:
def costo(asignacion):
    """Cuenta cuántos choques hay entre mentores en cada slot."""
    ocupacion = [0] * num_slots
    choques = 0
    for bloque in asignacion.values():
        if bloque is not None:
            ocupacion[bloque] += 1
            ocupacion[bloque + 1] += 1
    for o in ocupacion:
        if o > 1:
            choques += o - 1  # solo se cuenta el exceso
    return choques

In [13]:
def get_neighbors(asignacion, df):
    """Genera vecinos cambiando el bloque de un solo mentor."""
    vecinos = []
    for mentor in asignacion:
        actuales = asignacion.copy()
        disponibles = get_valid_blocks(df[df['MentorID'] == mentor].iloc[0, 1:])
        for nuevo_bloque in disponibles:
            if nuevo_bloque != asignacion[mentor]:
                vecino = actuales.copy()
                vecino[mentor] = nuevo_bloque
                vecinos.append(vecino)
    return vecinos

## Hill Climbing

In [14]:
def hill_climb(df, max_iters=1000):
    current = initial_solution(df)
    current_cost = costo(current)

    for i in range(max_iters):
        vecinos = get_neighbors(current, df)
        if not vecinos:
            break
        mejor_vecino = min(vecinos, key=costo)
        mejor_costo = costo(mejor_vecino)

        if mejor_costo < current_cost:
            current, current_cost = mejor_vecino, mejor_costo
        else:
            break  # Óptimo local

        if current_cost == 0:
            break  # Solución perfecta

    return current, current_cost

## Ejecución

In [15]:
asignacion_final, choques_finales = hill_climb(df)

## Reporte

In [16]:
print(f"Choques finales: {choques_finales}")
print("Asignación de bloques (slot inicial):")
for mentor, bloque in asignacion_final.items():
    if bloque is not None:
        print(f"{mentor}: Slot{bloque + 1} y Slot{bloque + 2}")
    else:
        print(f"{mentor}: ❌ Sin bloque válido")

Choques finales: 26
Asignación de bloques (slot inicial):
M01: Slot9 y Slot10
M02: Slot1 y Slot2
M03: Slot4 y Slot5
M04: Slot7 y Slot8
M05: Slot1 y Slot2
M06: Slot5 y Slot6
M07: Slot6 y Slot7
M08: Slot8 y Slot9
M09: Slot5 y Slot6
M10: Slot3 y Slot4
M11: ❌ Sin bloque válido
M12: Slot1 y Slot2
M13: Slot1 y Slot2
M14: Slot7 y Slot8
M15: Slot2 y Slot3
M16: Slot1 y Slot2
M17: Slot2 y Slot3
M18: Slot7 y Slot8
M19: Slot8 y Slot9
M20: ❌ Sin bloque válido
